## Importando as bibliotecas necessárias

In [85]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.utils import resample

import json

## Carregando a base de dados

In [6]:
query = "SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` WHERE date BETWEEN '20170101' AND '20170131'"

In [7]:
df = pd.read_gbq(query, project_id='still-function-331703', dialect='standard', dtypes={'visitId': object, 'date': object})

Requesting query... ok.
Job ID: 140e7194-b9fd-42a3-85a5-0ce789675f50
Query running...
  Elapsed 8.0 s. Waiting...
  Elapsed 9.07 s. Waiting...
  Elapsed 10.16 s. Waiting...
  Elapsed 11.25 s. Waiting...
  Elapsed 12.36 s. Waiting...
  Elapsed 13.47 s. Waiting...
  Elapsed 14.57 s. Waiting...
  Elapsed 15.69 s. Waiting...
  Elapsed 16.75 s. Waiting...
  Elapsed 17.84 s. Waiting...
  Elapsed 18.92 s. Waiting...
  Elapsed 20.0 s. Waiting...
  Elapsed 21.08 s. Waiting...
  Elapsed 22.18 s. Waiting...
  Elapsed 23.29 s. Waiting...
  Elapsed 24.37 s. Waiting...
  Elapsed 25.47 s. Waiting...
Query done.
Processed: 5.4 GB Billed: 5.4 GB
Standard price: $0.03 USD

Retrieving results...
Got 64694 rows.

Total time taken 509.75 s.
Finished at 2021-11-13 21:02:31.


In [28]:
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,clientId,channelGrouping,socialEngagementType
0,None,1,1485186621,1485186621,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...","[{'index': 4, 'value': 'APAC'}]","[{'product': [], 'isSecure': None, 'eCommerceA...",5506808496436529067,None,None,Organic Search,Not Socially Engaged
1,None,1,1485186152,1485186152,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...",[],"[{'product': [], 'isSecure': None, 'eCommerceA...",9647763979853103070,None,None,Organic Search,Not Socially Engaged
2,None,2,1485171966,1485171966,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...",[],"[{'product': [], 'isSecure': None, 'eCommerceA...",5674016882836571295,None,None,Organic Search,Not Socially Engaged
3,None,1,1485194986,1485194986,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'London', 'longitude': 'not availabl...","[{'index': 4, 'value': 'EMEA'}]","[{'product': [], 'isSecure': None, 'eCommerceA...",6657589312186007604,None,None,Direct,Not Socially Engaged
4,None,1,1485243569,1485243569,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': '(not set)', 'longitude': 'not avail...","[{'index': 4, 'value': 'EMEA'}]","[{'product': [], 'isSecure': None, 'eCommerceA...",1420011108511305656,None,None,Organic Search,Not Socially Engaged


In [13]:
df.shape

(64694, 16)

In [26]:
lst = set()
for x in df.hits:
    lst.add(len(x))
print(lst)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 97, 98, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 114, 116, 117, 118, 119, 122, 124, 125, 126, 127, 128, 130, 138, 140, 141, 142, 143, 146, 154, 161, 164, 167, 168, 174, 183, 188, 189, 191, 217, 227, 237, 303, 483, 500}


In [29]:
df.hits[0][0]

{'appInfo': {'appId': None,
  'appInstallerId': None,
  'appName': None,
  'appVersion': None,
  'exitScreenName': 'www.googlemerchandisestore.com/home',
  'id': None,
  'installerId': None,
  'landingScreenName': 'www.googlemerchandisestore.com/home',
  'name': None,
  'screenDepth': '0',
  'screenName': 'www.googlemerchandisestore.com/home',
  'version': None},
 'contentGroup': {'contentGroup1': '(not set)',
  'contentGroup2': '(not set)',
  'contentGroup3': '(not set)',
  'contentGroup4': '(not set)',
  'contentGroup5': '(not set)',
  'contentGroupUniqueViews1': None,
  'contentGroupUniqueViews2': None,
  'contentGroupUniqueViews3': None,
  'contentGroupUniqueViews4': None,
  'contentGroupUniqueViews5': None,
  'previousContentGroup1': '(entrance)',
  'previousContentGroup2': '(entrance)',
  'previousContentGroup3': '(entrance)',
  'previousContentGroup4': '(entrance)',
  'previousContentGroup5': '(entrance)'},
 'contentInfo': None,
 'customDimensions': [],
 'customMetrics': [],
 'c

In [30]:
produtos_sessao = []
precos_sessao = []

for linha in df.hits:
  produtos_hit = []
  precos_hit = []
  for hit in linha:
    for produto in hit['product']:
      produtos_hit.append(produto['productSKU'])
      precos_hit.append(produto['productPrice'])
  produtos_sessao.append(produtos_hit)
  precos_sessao.append(sum(precos_hit))

In [37]:
produtos_sessao = pd.Series(produtos_sessao)

In [42]:
mlb = MultiLabelBinarizer()
produtos_sessao_mlb = mlb.fit_transform(produtos_sessao)
produtos_sessao_mlb = pd.DataFrame(produtos_sessao_mlb, columns=mlb.classes_, index=df.index)
df = df.join(produtos_sessao_mlb)
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,...,GGOEYFKQ020699,GGOEYHPA003510,GGOEYHPA003610,GGOEYHPB072210,GGOEYOBR078599,GGOEYOCR077399,GGOEYOCR077799,GGOEYOCR078099,GGOEYOLR018699,GGOEYOLR080599
0,None,1,1485186621,1485186621,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...","[{'index': 4, 'value': 'APAC'}]",...,0,0,0,0,0,0,0,0,0,0
1,None,1,1485186152,1485186152,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...",[],...,0,0,0,0,0,0,0,0,0,0
2,None,2,1485171966,1485171966,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...",[],...,0,0,0,0,0,0,0,0,0,0
3,None,1,1485194986,1485194986,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'London', 'longitude': 'not availabl...","[{'index': 4, 'value': 'EMEA'}]",...,0,0,0,0,0,0,0,0,0,0
4,None,1,1485243569,1485243569,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': '(not set)', 'longitude': 'not avail...","[{'index': 4, 'value': 'EMEA'}]",...,0,0,0,0,0,0,0,0,0,0


In [43]:
df['precos'] = precos_sessao
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,...,GGOEYHPA003510,GGOEYHPA003610,GGOEYHPB072210,GGOEYOBR078599,GGOEYOCR077399,GGOEYOCR077799,GGOEYOCR078099,GGOEYOLR018699,GGOEYOLR080599,precos
0,None,1,1485186621,1485186621,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...","[{'index': 4, 'value': 'APAC'}]",...,0,0,0,0,0,0,0,0,0,0
1,None,1,1485186152,1485186152,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...",[],...,0,0,0,0,0,0,0,0,0,0
2,None,2,1485171966,1485171966,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...",[],...,0,0,0,0,0,0,0,0,0,0
3,None,1,1485194986,1485194986,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'London', 'longitude': 'not availabl...","[{'index': 4, 'value': 'EMEA'}]",...,0,0,0,0,0,0,0,0,0,0
4,None,1,1485243569,1485243569,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': '(not set)', 'longitude': 'not avail...","[{'index': 4, 'value': 'EMEA'}]",...,0,0,0,0,0,0,0,0,0,0


In [52]:
visitas = df.drop(['hits','customDimensions'], axis=1)
visitas.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,fullVisitorId,...,GGOEYHPA003510,GGOEYHPA003610,GGOEYHPB072210,GGOEYOBR078599,GGOEYOCR077399,GGOEYOCR077799,GGOEYOCR078099,GGOEYOLR018699,GGOEYOLR080599,precos
0,None,1,1485186621,1485186621,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...",5506808496436529067,...,0,0,0,0,0,0,0,0,0,0
1,None,1,1485186152,1485186152,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...",9647763979853103070,...,0,0,0,0,0,0,0,0,0,0
2,None,2,1485171966,1485171966,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'not available in demo dataset', 'lo...",5674016882836571295,...,0,0,0,0,0,0,0,0,0,0
3,None,1,1485194986,1485194986,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': 'London', 'longitude': 'not availabl...",6657589312186007604,...,0,0,0,0,0,0,0,0,0,0
4,None,1,1485243569,1485243569,20170123,"{'transactions': None, 'visits': 1, 'timeOnScr...","{'keyword': None, 'source': '(direct)', 'mediu...",{'mobileInputSelector': 'not available in demo...,"{'metro': '(not set)', 'longitude': 'not avail...",1420011108511305656,...,0,0,0,0,0,0,0,0,0,0


In [53]:
dicionarios = ['device', 'trafficSource', 'geoNetwork','totals']

# Transformando as chaves dos dicionários em novas colunas
for coluna in dicionarios:
    visitas = visitas.join(pd.DataFrame([json.loads(json.dumps(linha)) for linha in visitas[coluna]]),rsuffix=('_'+ coluna))
visitas.drop(dicionarios, axis=1, inplace=True)


# Corrigindo o formato das colunas com valores quantitativos
totals = df.totals[0].keys()
totals = list(totals)
for coluna in totals:
    visitas[coluna] = pd.to_numeric(visitas[coluna])


# Limpando os dados
visitas.drop('adwordsClickInfo',axis=1,inplace=True)
coluna_na = []
for coluna in visitas.columns:
    print(coluna + ': ' + str(len(visitas[coluna].unique())))
    if len(visitas[coluna].unique()) == 1:
        coluna_na.append(coluna)
visitas.drop(coluna_na,axis=1,inplace=True)

visitorId: 1
visitNumber: 202
visitId: 63585
visitStartTime: 63640
date: 31
fullVisitorId: 53041
userId: 1
clientId: 1
channelGrouping: 8
socialEngagementType: 1
10 52048: 2
10 52147: 2
10 52211: 2
10 52213: 2
10 52223: 2
10 52225: 2
10 52230: 2
10 52231: 2
10 52232: 2
10 52233: 2
10 52234: 2
10 52237: 2
10 52238: 2
10 52242: 2
10 52246: 2
10 52250: 2
10 52251: 2
9180748: 2
9180749: 2
9180750: 2
9180751: 2
9180752: 2
9180754: 2
9180755: 2
9180756: 2
9180757: 2
9180758: 2
9180759: 2
9180760: 2
9180761: 2
9180764: 2
9180766: 2
9180767: 2
9180769: 2
9180770: 2
9180771: 2
9180778: 2
9180779: 2
9180781: 2
9180782: 2
9180785: 2
9180791: 2
9180792: 2
9180793: 2
9180797: 2
9180801: 2
9180802: 2
9180804: 2
9180806: 2
9180808: 2
9180809: 2
9180811: 2
9180812: 2
9180814: 2
9180815: 2
9180816: 2
9180817: 2
9180818: 2
9180819: 2
9180820: 2
9180822: 2
9180824: 2
9180833: 2
9180834: 2
9180837: 2
9180838: 2
9180840: 2
9180841: 2
9180844: 2
9180850: 2
9180862: 2
9180863: 2
9180875: 2
9180905: 2
9181019

GGOEGAAX0617: 2
GGOEGAAX0618: 2
GGOEGAAX0619: 2
GGOEGAAX0620: 2
GGOEGAAX0621: 2
GGOEGAAX0622: 2
GGOEGAAX0623: 2
GGOEGAAX0624: 2
GGOEGAAX0625: 2
GGOEGAAX0626: 2
GGOEGAAX0627: 2
GGOEGAAX0628: 2
GGOEGAAX0651: 2
GGOEGAAX0652: 2
GGOEGAAX0653: 2
GGOEGAAX0654: 2
GGOEGAAX0655: 2
GGOEGAAX0656: 2
GGOEGAAX0657: 2
GGOEGAAX0659: 2
GGOEGAAX0660: 2
GGOEGAAX0661: 2
GGOEGAAX0662: 2
GGOEGAAX0663: 2
GGOEGAAX0664: 2
GGOEGAAX0671: 2
GGOEGAAX0679: 2
GGOEGAAX0680: 2
GGOEGAAX0681: 2
GGOEGAAX0682: 2
GGOEGAAX0683: 2
GGOEGAAX0684: 2
GGOEGAAX0685: 2
GGOEGAAX0686: 2
GGOEGAAX0687: 2
GGOEGAAX0688: 2
GGOEGAAX0689: 2
GGOEGAAX0690: 2
GGOEGAAX0729: 2
GGOEGAAX0730: 2
GGOEGAAX0731: 2
GGOEGAAX0732: 2
GGOEGAAX0733: 2
GGOEGAAX0734: 2
GGOEGAAX0735: 2
GGOEGAAX0746: 2
GGOEGAAX0781: 2
GGOEGAAX0795: 2
GGOEGACB023699: 2
GGOEGADB056614: 2
GGOEGADB056615: 2
GGOEGADB056616: 2
GGOEGADB056617: 2
GGOEGADB056618: 2
GGOEGADB056713: 2
GGOEGADB056714: 2
GGOEGADB056715: 2
GGOEGADB056716: 2
GGOEGADB056717: 2
GGOEGADB056914: 2
GGOEGADB056915: 

GGOEYAEJ029515: 2
GGOEYAEJ029517: 2
GGOEYAEJ029616: 2
GGOEYAEJ033514: 2
GGOEYAEJ033515: 2
GGOEYAEJ033516: 2
GGOEYAFB073113: 2
GGOEYAFB073114: 2
GGOEYAFB073115: 2
GGOEYAFB073116: 2
GGOEYAFB073117: 2
GGOEYAQB073213: 2
GGOEYAQB073214: 2
GGOEYAQB073215: 2
GGOEYAQB073217: 2
GGOEYAWJ061448: 2
GGOEYAWJ061449: 2
GGOEYAWR062349: 2
GGOEYAWR062350: 2
GGOEYAXR066128: 2
GGOEYAXR066129: 2
GGOEYAXR066130: 2
GGOEYAYR068624: 2
GGOEYAYR068625: 2
GGOEYAYR068626: 2
GGOEYAYR068656: 2
GGOEYDHJ019399: 2
GGOEYDHJ056099: 2
GGOEYFKQ020699: 2
GGOEYHPA003510: 2
GGOEYHPA003610: 2
GGOEYHPB072210: 2
GGOEYOBR078599: 2
GGOEYOCR077399: 2
GGOEYOCR077799: 2
GGOEYOCR078099: 2
GGOEYOLR018699: 2
GGOEYOLR080599: 2
precos: 13589
browser: 27
browserSize: 1
browserVersion: 1
deviceCategory: 3
flashVersion: 1
isMobile: 2
javaEnabled: 1
language: 1
mobileDeviceBranding: 1
mobileDeviceInfo: 1
mobileDeviceMarketingName: 1
mobileDeviceModel: 1
mobileInputSelector: 1
operatingSystem: 15
operatingSystemVersion: 1
screenColors: 1
scree

In [54]:
visitas.head()

,visitNumber,visitId,visitStartTime,date,fullVisitorId,channelGrouping,10 52048,10 52147,10 52211,10 52213,...,region,subContinent,bounces,hits,newVisits,pageviews,timeOnSite,totalTransactionRevenue,transactionRevenue,transactions
0,1,1485186621,1485186621,20170123,5506808496436529067,Organic Search,0,0,0,0,...,not available in demo dataset,Eastern Asia,1.0,1,1.0,1.0,NaN,NaN,NaN,NaN
1,1,1485186152,1485186152,20170123,9647763979853103070,Organic Search,0,0,0,0,...,not available in demo dataset,Western Africa,1.0,1,1.0,1.0,NaN,NaN,NaN,NaN
2,2,1485171966,1485171966,20170123,5674016882836571295,Organic Search,0,0,0,0,...,not available in demo dataset,Western Asia,1.0,1,NaN,1.0,NaN,NaN,NaN,NaN
3,1,1485194986,1485194986,20170123,6657589312186007604,Direct,0,0,0,0,...,England,Northern Europe,1.0,1,1.0,1.0,NaN,NaN,NaN,NaN
4,1,1485243569,1485243569,20170123,1420011108511305656,Organic Search,0,0,0,0,...,(not set),Northern Europe,1.0,1,1.0,1.0,NaN,NaN,NaN,NaN


In [55]:
visitas.drop(['transactionRevenue', 'transactions'],axis=1,inplace=True)
visitas.totalTransactionRevenue = visitas.totalTransactionRevenue / 1000000

In [56]:
# Criando lista com variaveis quantitativas
quant = list(set(totals) - set(coluna_na) - set(['transactionRevenue','transactions']))

#Criando dataframe com os resultadas quantitativos das sessoes
visitas_totals = visitas.groupby('fullVisitorId',as_index=False)[quant].sum()

# Última visita
visitas_ultima = visitas.groupby('fullVisitorId',as_index=False)
visitas_ultima = visitas_ultima['visitNumber'].max()

# Combinação entre visitantes e visitas únicos
usuarios_visitas_unicos = visitas.drop_duplicates(subset=['fullVisitorId','visitNumber'])

# Dataframe usuários com todos os usuários unicos e sua última visita
usuarios = pd.merge(visitas_ultima,usuarios_visitas_unicos,left_on=['fullVisitorId','visitNumber'],
                    right_on=['fullVisitorId','visitNumber'],how='left')

# Primeira visita
visitas_primeira = visitas.groupby('fullVisitorId',as_index=False)['visitNumber'].min()
visitas_primeira.set_index('fullVisitorId',inplace=True)

# Dataframe usuários com todos os usuários unicos + sua última visita + primeira visita
usuarios = usuarios.join(visitas_primeira,how='left',on='fullVisitorId',rsuffix='_primeira')
usuarios = pd.merge(usuarios,usuarios_visitas_unicos,left_on=['fullVisitorId','visitNumber_primeira'],
                    right_on=['fullVisitorId','visitNumber'],how='left',suffixes=('_ultima','_primeira'))

# Dataframe usuários com todos os usuários unicos + sua última visita + primeira visita + somatório das colunas quant
usuarios = pd.merge(usuarios,visitas_totals,left_on=['fullVisitorId'],
                    right_on=['fullVisitorId'],how='left')

# Removendo totais
for i in quant:
    usuarios.drop(i+'_primeira',axis=1,inplace=True)
    usuarios.drop(i+'_ultima',axis=1,inplace=True)

# Calculando o tempo entre primeira e última visista
usuarios['tempo_visitas'] = usuarios.visitStartTime_ultima - usuarios.visitStartTime_primeira

#Removendo as colunas Ids
ids = ['fullVisitorId', 'visitId_ultima','visitId_primeira']
usuarios.drop(ids,axis=1,inplace=True)

#Criando uma variavel y com a coluna resposta
y = usuarios.totalTransactionRevenue.copy()
y.fillna(0,inplace=True)
y[y<0] = 0

#Criando uma variavel X com todas as variaveis menos a resposta
X = usuarios.drop('totalTransactionRevenue',axis=1)


#Transformando as variaveis qualitativas em numeros
from sklearn.preprocessing import LabelEncoder
quali = usuarios.dtypes[usuarios.dtypes == 'object'].keys()
for col in quali:
    lbl = LabelEncoder()
    lbl.fit(list(X[col].values.astype('str')))
    X[col] = lbl.transform(list(X[col].values.astype('str')))

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [60]:
X.shape

(53041, 2271)

In [72]:
X = X.loc[:,~X.columns.duplicated()]

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Modelo 01 - Regressão linear

In [77]:
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred_reg = reg.predict(X_test) 
print(np.sqrt(mean_squared_error(y_test, y_pred_reg)))

69.41644592279307


## Modelo 02 - Gradient Boosting Regressor

In [81]:
gb = GradientBoostingRegressor()
gb.fit(X_train,y_train)
y_gb_predict = gb.predict(X_test)
y_gb_predict[y_gb_predict < 0] = 0
print(np.sqrt(mean_squared_error(y_test, y_gb_predict)))

41.0844537567586


## Modelo Dummy

In [80]:
dr = DummyRegressor()
dr.fit(X_train, y_train)
y_dr_predict = dr.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_dr_predict))

43.006562211198855

## Modelo 03 - Mudando a estratêgia, prevendo se o usuario consumiu ou não

In [83]:
y_train.values[y_train <= 0] = 0
y_train.values[y_train > 0] = 1
y_test.values[y_test <= 0] = 0
y_test.values[y_test > 0] = 1

In [84]:
clf_gb = GradientBoostingClassifier()
clf_gb.fit(X_train, y_train)
clf_gb_predict = clf_gb.predict(X_test)
print(confusion_matrix(y_test,clf_gb_predict))
print(classification_report(y_test,clf_gb_predict))

[[15649    61]
 [  126    77]]
             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     15710
        1.0       0.56      0.38      0.45       203

avg / total       0.99      0.99      0.99     15913



## Modelo 04 - Downsampling

In [88]:
X_train['revenue'] = y_train
usuarios_com_gastos = X_train[X_train.revenue > 0]
usuarios_sem_gastos = X_train[X_train.revenue <= 0]

usuarios_sem_gastos_ds = resample(usuarios_sem_gastos,
                                  replace=False,
                                  n_samples=30000,
                                  random_state=42)

X_train_ds = pd.concat([usuarios_sem_gastos_ds, usuarios_com_gastos])

y_train_ds = X_train_ds.revenue.copy()
X_train_ds.drop('revenue',axis=1,inplace=True)

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [89]:
clf_gb_ds = GradientBoostingClassifier(random_state=42)
clf_gb_ds.fit(X_train_ds, y_train_ds)
y_pred_clf_gb_predict_ds = clf_gb_ds.predict(X_test)

print(confusion_matrix(y_test,y_pred_clf_gb_predict_ds))
print(classification_report(y_test,y_pred_clf_gb_predict_ds))

[[15643    67]
 [  118    85]]
             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     15710
        1.0       0.56      0.42      0.48       203

avg / total       0.99      0.99      0.99     15913



## Modelo 05 - XGBoost

In [90]:
xgb = XGBClassifier(seed=42,n_estimators=300)
X_test = X_test.loc[:,~X_test.columns.duplicated()]
xgb.fit(X_train_ds, y_train_ds)
y_pred_clf_xgb_predict_ds = xgb.predict(X_test)

print(confusion_matrix(y_test,y_pred_clf_xgb_predict_ds))
print(classification_report(y_test,y_pred_clf_xgb_predict_ds))

[[15655    55]
 [   95   108]]
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     15710
        1.0       0.66      0.53      0.59       203

avg / total       0.99      0.99      0.99     15913



/usr/local/envs/py3env/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Modelo 06 - XGBoost com diferente downsampling

In [99]:
X_train['revenue'] = y_train
usuarios_com_gastos = X_train[X_train.revenue > 0]
usuarios_sem_gastos = X_train[X_train.revenue <= 0]

usuarios_sem_gastos_ds = resample(usuarios_sem_gastos,
                                  replace=False,
                                  n_samples=25000,
                                  random_state=42)

X_train_ds = pd.concat([usuarios_sem_gastos_ds, usuarios_com_gastos])

y_train_ds = X_train_ds.revenue.copy()
X_train_ds.drop('revenue',axis=1,inplace=True)

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [100]:
xgb = XGBClassifier(seed=42,n_estimators=300)
X_test = X_test.loc[:,~X_test.columns.duplicated()]
xgb.fit(X_train_ds, y_train_ds)
y_pred_clf_xgb_predict_ds = xgb.predict(X_test)

print(confusion_matrix(y_test,y_pred_clf_xgb_predict_ds))
print(classification_report(y_test,y_pred_clf_xgb_predict_ds))

[[15640    70]
 [   90   113]]
             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     15710
        1.0       0.62      0.56      0.59       203

avg / total       0.99      0.99      0.99     15913



/usr/local/envs/py3env/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
